In [3]:
# !pip install yfinance --upgrade --no-cache-dir
# !pip install beautifulsoup4

In [1]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np
import io
# from google.colab import files
import os
import datetime
import time
import sys
import math
#from lxml import etree
from io import StringIO, BytesIO
import seaborn as sns
import yfinance as yf

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
ten_q_with_ticker_7 = pd.read_csv('ten_q_with_ticker/ten_q_with_ticker_7.csv')

In [4]:
def obtain_mda_and_prices_data(df):
    current_set = df
    for index, row in current_set.iterrows():
        try:
              
            starttime = datetime.datetime.now()
            # print("Processing row number", index)
            
            ###### PARSING 10Q SECTION TO EXTRACT MDA ######
            ten_q_raw = requests.get(row["sec_url"]).text

            ## parse the raw text and get rid of tags
            soup = BeautifulSoup(ten_q_raw, 'lxml')
            notags = soup.getText()

            ## use regex to extract MDA
            #### first splitting on all mentions of "Item 2. Management's Discussion and Analysis" ignoring case sensitivity
            #### and taking the last element of split list
            start_of_mda = re.split("Item[\s]*2[\s]*[.\-–:]+[\s]*[.\-–]*[\s]*Management[\S]?s?[\s]*Discussion[\s]*and[\s]*Analysis[\s]*", 
                          notags, 
                          flags = re.IGNORECASE)[-1]
                    
            #### then taking that last element from above and splitting on "Item 3. Quantitative and Qualitative Disclosures About Market Risk" ignoring case sensitivity
            #### and taking the first element of this split list to get the full MDA section
            full_mda = re.split("Item[\s]*[34][\s]*[.\-–:]*[\s]*[.\-–]*[\s]*(Quantitative[\s]*and[\s]*Qualitative[\s]*disclosures[\s]*|Controls[\s]*and[\s]*Procedures[\s]*)", 
                                start_of_mda, 
                                flags = re.IGNORECASE)[0]

            mda_length = len(full_mda)
            full_ten_q_length = len(notags)
            mda_percentage = mda_length/full_ten_q_length

            current_set.loc[index, 'mda_raw_text'] = full_mda
            current_set.loc[index, 'ending_of_mda'] = full_mda[-100:-1]
            current_set.loc[index, 'mda_length'] = mda_length
            current_set.loc[index, 'full_ten_q_length'] = full_ten_q_length
            current_set.loc[index, 'mda_percentage'] = mda_percentage
            endtime = datetime.datetime.now()

            # print("Current row took", endtime - starttime)

            ###### (COMPLETED) PARSING 10Q SECTION TO EXTRACT MDA ######



            ####### OBTAINING STOCK PRICE DATA #######
            #### closing price on filing date and opening price on next trading day
            row_ticker = row["ticker"]
            row_date = datetime.datetime.strptime(row["accepted_filing_date"], "%Y-%m-%d").date() 
            prices_for_timewindow = yf.Ticker(row_ticker).history(start = row_date, end = row_date + datetime.timedelta(days = 6)).reset_index() ## setting time window to 6 days because this will ensure that we get two consecutive trading days (avoiding weekends and holidays)
            prices_for_timewindow = prices_for_timewindow.iloc[:2,:]

            if prices_for_timewindow.shape[0] > 1:          ## only fill in values in columns if pricing data is available for these dates
                current_set.loc[index, "trading_date_closest_to_filing_date"] = prices_for_timewindow.loc[0,"Date"] 
                current_set.loc[index, "closing_price_on_filing_date"] = prices_for_timewindow.loc[0,"Close"]

                current_set.loc[index, "next_trading_date"] = prices_for_timewindow.loc[1,"Date"]
                current_set.loc[index, "opening_price_on_next_trading_date"] = prices_for_timewindow.loc[1,"Open"]


            #### opening price on x days later
            days_after = [7,30]
            for window in days_after:
                prices_for_timewindow = yf.Ticker(row_ticker).history(start = row_date, end = row_date + datetime.timedelta(days = window + 1)).reset_index()
                prices_for_timewindow_rows = prices_for_timewindow.shape[0]

                if prices_for_timewindow_rows > 1:         ## also only fill in values in columns if pricing data is available
                    current_set.loc[index, "trading_date_" + str(window) + "_days_after"] = prices_for_timewindow.loc[prices_for_timewindow_rows - 1,"Date"]
                    current_set.loc[index, "opening_price_" + str(window) + "_days_after"] = prices_for_timewindow.loc[prices_for_timewindow_rows - 1,"Open"]

            ####### (COMPLETED) OBTAINING STOCK PRICE DATA #######


            # no need to sleep because each iteration takes about 1.5 seconds (and the limit is 10 calls per second)
            time.sleep(0.1)
            if index%100 == 0:
                print("Just finished row number", index)
                print(datetime.datetime.now())
                print()
        except:
            print("Hit an error while processing row", str(index))
            print("Continuing to next row")
            continue

    return current_set

In [5]:
def chunk_process_data_and_write(df, subset, chunks):
    rows_in_df = df.shape[0]
    chunk_size = math.floor(rows_in_df / chunks)

    for i in range(chunks):
        if i != chunks - 1:
            starting = i * chunk_size
            ending = chunk_size * (i+1)
        else:
            starting = i * chunk_size
            ending = -1

        next_dataset_to_process = df[starting:ending]
        output = obtain_mda_and_prices_data(next_dataset_to_process)
        
        print("Writing to CSV: " + 'data_augmented_' + str(subset) + '_' + str(starting) + 'to' + str(ending) + '.csv')
        output.to_csv('data_augmented_' + str(subset) + '_' + str(starting) + 'to' + str(ending) + '.csv', index = False)

In [6]:
chunk_process_data_and_write(ten_q_with_ticker_7, 7, 10)

Just finished row number 0
2021-02-21 23:33:03.814109

- WVVIP: Data doesn't exist for startDate = 1463025600, endDate = 1463544000
- WVVIP: Data doesn't exist for startDate = 1463025600, endDate = 1463716800
- WVVIP: Data doesn't exist for startDate = 1463025600, endDate = 1465704000
- WVVIP: Data doesn't exist for startDate = 1470888000, endDate = 1471406400
- WVVIP: Data doesn't exist for startDate = 1470888000, endDate = 1471579200
- JAG: No data found, symbol may be delisted
- JAG: No data found, symbol may be delisted
- JAG: No data found, symbol may be delisted
- JAG: No data found, symbol may be delisted
- JAG: No data found, symbol may be delisted
- JAG: No data found, symbol may be delisted
Just finished row number 100
2021-02-21 23:36:50.888508

- JAG: No data found, symbol may be delisted
- JAG: No data found, symbol may be delisted
- JAG: No data found, symbol may be delisted
- JAG: No data found, symbol may be delisted
- JAG: No data found, symbol may be delisted
- JAG: N

- AMAR: Data doesn't exist for startDate = 1479704400, endDate = 1480222800
- AMAR: Data doesn't exist for startDate = 1479704400, endDate = 1480395600
- AMAR: Data doesn't exist for startDate = 1479704400, endDate = 1482382800
- AMAR: Data doesn't exist for startDate = 1526961600, endDate = 1527480000
- AMAR: Data doesn't exist for startDate = 1526961600, endDate = 1527652800
- AMAR: Data doesn't exist for startDate = 1526961600, endDate = 1529640000
- AMAR: Data doesn't exist for startDate = 1565323200, endDate = 1565841600
- AMAR: Data doesn't exist for startDate = 1565323200, endDate = 1566014400
- AMAR: Data doesn't exist for startDate = 1565323200, endDate = 1568001600
- HLM-P: No data found for this date range, symbol may be delisted
- HLM-P: No data found for this date range, symbol may be delisted
- HLM-P: No data found for this date range, symbol may be delisted
- HLM-P: No data found for this date range, symbol may be delisted
- HLM-P: No data found for this date range, symb

- GPIC: No data found, symbol may be delisted
- GPIC: No data found, symbol may be delisted
- GPIC: No data found, symbol may be delisted
- GPIC: No data found, symbol may be delisted
- GPIC: No data found, symbol may be delisted
- GPIC: No data found, symbol may be delisted
- GPIC: No data found, symbol may be delisted
- GPIC: No data found, symbol may be delisted
- GPIC: No data found, symbol may be delisted
- GPIC: No data found, symbol may be delisted
- GPIC: No data found, symbol may be delisted
- GPIC: No data found, symbol may be delisted
- GPIC: No data found, symbol may be delisted
- GPIC: No data found, symbol may be delisted
- GPIC: No data found, symbol may be delisted
- GPIC: No data found, symbol may be delisted
- GPIC: No data found, symbol may be delisted
- GPIC: No data found, symbol may be delisted
- GPIC: No data found, symbol may be delisted
- GPIC: No data found, symbol may be delisted
- GPIC: No data found, symbol may be delisted
- GPIC: No data found, symbol may 

- MADL: No data found, symbol may be delisted
- MADL: No data found, symbol may be delisted
- MADL: No data found, symbol may be delisted
- MADL: No data found, symbol may be delisted
- MADL: No data found, symbol may be delisted
- MADL: No data found, symbol may be delisted
- MADL: No data found, symbol may be delisted
- MADL: No data found, symbol may be delisted
- MADL: No data found, symbol may be delisted
- MADL: No data found, symbol may be delisted
- MADL: No data found, symbol may be delisted
- MADL: No data found, symbol may be delisted
- MADL: No data found, symbol may be delisted
- MADL: No data found, symbol may be delisted
- MADL: No data found, symbol may be delisted
Just finished row number 1000
2021-02-22 00:24:28.569924

- CSBB: Data doesn't exist for startDate = 1502424000, endDate = 1502942400
- CSBB: Data doesn't exist for startDate = 1502424000, endDate = 1503115200
- CSBB: Data doesn't exist for startDate = 1502424000, endDate = 1505102400
- CSBB: Data doesn't exi

- MDWT: Data doesn't exist for startDate = 1463112000, endDate = 1463630400
- MDWT: Data doesn't exist for startDate = 1463112000, endDate = 1463803200
- MDWT: Data doesn't exist for startDate = 1463112000, endDate = 1465790400
- MDWT: Data doesn't exist for startDate = 1470974400, endDate = 1471492800
- MDWT: Data doesn't exist for startDate = 1470974400, endDate = 1471665600
- MDWT: Data doesn't exist for startDate = 1470974400, endDate = 1473652800
- MDWT: Data doesn't exist for startDate = 1597377600, endDate = 1597896000
- MDWT: Data doesn't exist for startDate = 1597377600, endDate = 1598068800
- MDWT: Data doesn't exist for startDate = 1597377600, endDate = 1600056000
- MDWT: Data doesn't exist for startDate = 1494561600, endDate = 1495080000
- MDWT: Data doesn't exist for startDate = 1494561600, endDate = 1495252800
- MDWT: Data doesn't exist for startDate = 1494561600, endDate = 1497240000
- MDWT: Data doesn't exist for startDate = 1541739600, endDate = 1542258000
- MDWT: Data

...

- AGS: Data doesn't exist for startDate = 1494388800, endDate = 1494907200
- AGS: Data doesn't exist for startDate = 1494388800, endDate = 1495080000
- AGS: Data doesn't exist for startDate = 1494388800, endDate = 1497067200
- AGS: Data doesn't exist for startDate = 1479099600, endDate = 1479618000
- AGS: Data doesn't exist for startDate = 1479099600, endDate = 1479790800
- AGS: Data doesn't exist for startDate = 1479099600, endDate = 1481778000
- GCEA: No data found for this date range, symbol may be delisted
- GCEA: No data found for this date range, symbol may be delisted
- GCEA: No data found for this date range, symbol may be delisted
- GCEA: No data found for this date range, symbol may be delisted
- GCEA: No data found for this date range, symbol may be delisted
- GCEA: No data found for this date range, symbol may be delisted
- GCEA: No data found for this date range, symbol may be delisted
- GCEA: No data found for this date range, symbol may be delisted
- GCEA: No data found 

- SIGY: Data doesn't exist for startDate = 1526356800, endDate = 1526875200
- SIGY: Data doesn't exist for startDate = 1526356800, endDate = 1527048000
- SIGY: Data doesn't exist for startDate = 1526356800, endDate = 1529035200
- SIGY: Data doesn't exist for startDate = 1564977600, endDate = 1565496000
- SIGY: Data doesn't exist for startDate = 1564977600, endDate = 1565668800
- SIGY: Data doesn't exist for startDate = 1564977600, endDate = 1567656000
- IMUCW: No data found for this date range, symbol may be delisted
- IMUCW: No data found for this date range, symbol may be delisted
- IMUCW: No data found for this date range, symbol may be delisted
- IMUC: Data doesn't exist for startDate = 1502683200, endDate = 1503201600
- IMUC: Data doesn't exist for startDate = 1502683200, endDate = 1503374400
- IMUC: Data doesn't exist for startDate = 1502683200, endDate = 1505361600
- IMUCW: No data found for this date range, symbol may be delisted
- IMUCW: No data found for this date range, symb

- XNDA: Data doesn't exist for startDate = 1589169600, endDate = 1589688000
- XNDA: Data doesn't exist for startDate = 1589169600, endDate = 1589860800
- XNDA: Data doesn't exist for startDate = 1589169600, endDate = 1591848000
- XNDA: Data doesn't exist for startDate = 1579669200, endDate = 1580187600
- XNDA: Data doesn't exist for startDate = 1579669200, endDate = 1580360400
- XNDA: Data doesn't exist for startDate = 1579669200, endDate = 1582347600
- XNDA: Data doesn't exist for startDate = 1579669200, endDate = 1580187600
- XNDA: Data doesn't exist for startDate = 1579669200, endDate = 1580360400
- XNDA: Data doesn't exist for startDate = 1579669200, endDate = 1582347600
- XNDA: Data doesn't exist for startDate = 1579669200, endDate = 1580187600
- XNDA: Data doesn't exist for startDate = 1579669200, endDate = 1580360400
- XNDA: Data doesn't exist for startDate = 1579669200, endDate = 1582347600
- XNDA: Data doesn't exist for startDate = 1579755600, endDate = 1580274000
- XNDA: Data

- HTBC: No data found for this date range, symbol may be delisted
- HTBC: No data found for this date range, symbol may be delisted
- HTBC: No data found for this date range, symbol may be delisted
- HTBC: No data found for this date range, symbol may be delisted
- HTBC: No data found for this date range, symbol may be delisted
- HTBC: No data found for this date range, symbol may be delisted
Just finished row number 8700
2021-02-22 08:43:56.564059

- PAYD: Data doesn't exist for startDate = 1463371200, endDate = 1463889600
- PAYD: Data doesn't exist for startDate = 1463371200, endDate = 1464062400
- PAYD: Data doesn't exist for startDate = 1463371200, endDate = 1466049600
- PAYD: Data doesn't exist for startDate = 1470974400, endDate = 1471492800
- PAYD: Data doesn't exist for startDate = 1470974400, endDate = 1471665600
- PAYD: Data doesn't exist for startDate = 1470974400, endDate = 1473652800
- PAYD: Data doesn't exist for startDate = 1479099600, endDate = 1479618000
- PAYD: Data d

- BEEMW: No data found for this date range, symbol may be delisted
- BEEMW: No data found for this date range, symbol may be delisted
- BEEMW: No data found for this date range, symbol may be delisted
- HGHH: No data found, symbol may be delisted
- HGHH: No data found, symbol may be delisted
- HGHH: No data found, symbol may be delisted
- HGHH: No data found, symbol may be delisted
- HGHH: No data found, symbol may be delisted
- HGHH: No data found, symbol may be delisted
- HGHH: No data found, symbol may be delisted
- HGHH: No data found, symbol may be delisted
- HGHH: No data found, symbol may be delisted
Writing to CSV: data_augmented_7_8091to-1.csv
